**Resources**
<!-- {{< youtube akZ8JJ4gGLs >}} -->
<!-- {{< tweet 1266579585417613312 >}} -->
[Smoothstep](https://smoothstep.io/)

- Breadth First Search
- Dijkstra's algorithm/Fast Marching Method for solving the Eikonal equation?
- BFS is generalized as Disjkstra, which is generalized as Fast Marching, then as Ordered Upwind method, then as Anisotropic Fast Marching

- Hex mazes
- Bellman-Ford Algorithm
- Fast Marching Algorithm (FMM), eikonal equation
- Art: https://troika.uk.com/work/troika-labyrinth/
- https://en.wikipedia.org/wiki/Lee_algorithm
- [Procedural Content Generation: Mazes](http://pcg.wikidot.com/pcg-algorithm:maze)
- [Wikipedia Maze Generation Algorithms](https://en.wikipedia.org/wiki/Maze_generation_algorithm)
- [Smart Move: Intelligent Path Finding](https://www.gamedeveloper.com/programming/smart-move-intelligent-path-finding)
- [Toward more Realistic Path Finding](https://www.gamedeveloper.com/programming/toward-more-realistic-pathfinding)
- [AI Wisdom A* Articles](http://www.aiwisdom.com/ai_astar.html)



Solving a Maze
- https://en.wikipedia.org/wiki/Maze-solving_algorithm


Per wikipedia:
> The best-known rule for traversing mazes is the wall follower, also known as either the left-hand rule or the right-hand rule. If the maze is simply connected, that is, all its walls are connected together or to the maze's outer boundary, then by keeping one hand in contact with one wall of the maze the solver is guaranteed not to get lost and will reach a different exit if there is one; otherwise, the algorithm will return to the entrance having traversed every corridor next to that connected section of walls at least once. The algorithm is a depth-first in-order tree traversal.

For autonomous agents I perhaps one of these will be suitable.
- [Maze Routing Algorithm](https://en.wikipedia.org/wiki/Maze-solving_algorithm#Maze-routing_algorithm)
- [Shortest Path Algorithms](https://en.wikipedia.org/wiki/Maze-solving_algorithm#Shortest_path_algorithm)


What do I really want to do here?
- Ultimately, I want to get back to working on the game. 
For NPCs:
  - Find the shortest distance between to points. Example enter/exit the maze.
  - Follow another character in the maze.
  - Search for an item in the maze.
  - animate moving a maze explorer
  - draw out the solution and possibly rejected solutions.
  - The AI should only have knowledge of the rooms its visited and the room it's
    currently in. Is that true though?

Questions
- What algorithm does Godot use?
  A* and 

Next Steps
- Try doing a simple animation using iPyCanvas. 
- Need a way to make an animated gif from the animation if I'm going to be able
  to put it on the blog.
- Agent that just walks the maze randomly.
- Agent that uses an algorithm for traversing.
- Multiple agents interacting (chasing, hiding)
- It may make sense to generate the same maze every time. Perhaps seed the random number generator. 

In [2]:
# Load python code.
%load_ext autoreload
%autoreload 2

In [88]:
# All imports
from __future__ import annotations

from heapq import heappop, heappush
from IPython.display import display
from ipycanvas import Canvas,  hold_canvas, MultiCanvas
import random
import time
from typing import Dict, List, Set, Tuple, Union
import itertools

from generation.structures import Corner, Point
from generation.maze import Maze, MazeCell
from generation.generators.random_backtracer import generate_maze_walls
from generation.npc import Agent
from generation.renderers.units import ROOM_SIZE_WIDTH, ROOM_SIZE_HEIGHT
from generation.renderers.wall_drawer import draw_maze, animate_drawing_by_rooms
from generation.renderers.agents_renderer import draw_agents
from generation.direction import Direction, DIR_ORIENTATION, Orientation

In [89]:
def find_next_direction(agent: Agent, possible_directions: list[Direction]) -> Direction:
  """The agent is facing a direction. I think it should continue in the same direction
  if possible. If not it should try right, then left. Finally, backtrack."""
  current_orientation: dict[Orientation, Direction] = DIR_ORIENTATION[agent.facing]
  if agent.facing in possible_directions:
    next_direction = agent.facing
  elif current_orientation[Orientation.RIGHT] in possible_directions: # Is there a door to the right?
    next_direction = current_orientation[Orientation.RIGHT]
  elif current_orientation[Orientation.LEFT]  in possible_directions: # Is there a door to the left?
    next_direction = current_orientation[Orientation.LEFT]
  elif current_orientation[Orientation.BEHIND] in possible_directions: # Go back the way we came?
    next_direction = current_orientation[Orientation.BEHIND]
  else:
    # This shouldn't be possible
    print('We\'re walled in!')
    print(f'Agent Facing: {agent.facing}')
    print(f'Possible Directions: {possible_directions}')
    print(f'Agent Orientation: {current_orientation}')
    raise Exception("We\'re walled in! No possible doors found.")
  return next_direction

# Traversal
def random_walk(agent: Agent, maze: Maze):
  """
  Given an agent, have them randomly choose a room to go to next, then move.
  """
  # 1. Get the current room the agent is in.
  current_room = maze.cell(agent.location)

  #  If the agent is at the entrance or exit of the maze, then stop.
  if current_room is maze.starting_cell or current_room is maze.exit_cell:
    return;

  # 2. Find all the walls that have doors in that room.
  possible_directions: List[Direction] = current_room.open_sides()

  # 3. Find the next direction to go.
  next_direction = find_next_direction(agent, possible_directions)

  # 4. Find the location of the room the open door connects to.
  next_location: Point = maze.find_adjacent_neighbor(next_direction, agent.location)

  # 5. Move the agent to the next room.
  agent.face(next_direction)
  agent.move_to(next_location)

def wall_follower_walk(agent: Agent, maze: Maze) -> None:
  """
  Maze traversal algorithm. Assumes that all the walls are connected.
  The agent keeps a 'hand' on the left or right wall. This means that when 
  deciding on the next move (if right dominate):
  1. Go right if possible.
  2. Else go forward if possible.
  3. Else go left if possible.
  4. Else go back if possible.
  5. Else panic.
  """
  # 1. Get the current room the agent is in.
  current_room = maze.cell(agent.location)

  # If the agent is at the entrance or exit of the maze, then stop.
  if current_room is maze.starting_cell or current_room is maze.exit_cell:
    return;

  # 2. Find the agent's current orientation.
  current_orientation: dict[Orientation, Direction] = DIR_ORIENTATION[agent.facing]

  # 3. Find all the walls that have doors in that room.
  possible_directions: List[Direction] = current_room.open_sides()

  # 4. Use the wall follower strategy to pick the next room.
  if current_orientation[Orientation.RIGHT] in possible_directions: # Is there a door to the right?
    next_direction = current_orientation[Orientation.RIGHT]
  elif agent.facing in possible_directions:
    next_direction = agent.facing
  elif current_orientation[Orientation.LEFT]  in possible_directions: # Is there a door to the left?
    next_direction = current_orientation[Orientation.LEFT]
  elif current_orientation[Orientation.BEHIND] in possible_directions: # Go back the way we came?
    next_direction = current_orientation[Orientation.BEHIND]
  else:
    # This shouldn't be possible
    print('We\'re walled in!')
    print(f'Agent Facing: {agent.facing}')
    print(f'Possible Directions: {possible_directions}')
    print(f'Agent Orientation: {current_orientation}')
    raise Exception("We\'re walled in! No possible doors found.")

  # 4. Find the location of the room the open door connects to.
  next_location: Point = maze.find_adjacent_neighbor(next_direction, agent.location)

  # 5. Move the agent to the next room.
  agent.face(next_direction)
  agent.move_to(next_location)

In [90]:
"""A Star Implementation"""

"""
- TODO: Add A* with the exit or entrance as target
- TODO: Make the Renderer class agnostic of what's being drawn.
- TODO: Blog about traversal? This seems redundant. What might be more interesting is NPC with path traversal. An exploration about Dwarf Fortress's NPCs.
- TODO: Try some other maze generation algorithms or go back to focusing on the game.
- TODO: Playing with Shaders in Godot
"""

"""
https://en.wikipedia.org/wiki/A*_search_algorithm
- Should look up path finding in the GDC youtube channel.
  - Videos on Hollow Knight?

The algorithm:
Start at the initial position (node) and place it on the Open list, 
along with its estimated cost to the destination, which is determined by a heuristic. 
The heuristic is often just the geometric distance between two nodes. 

Then perform the following loop while the Open list is nonempty:
- Pop the node off the Open list that has the lowest estimated cost to the destination.
- If the node is the destination, we've successfully finished (quit).
- Examine the node's neighboring nodes.
- For each of the nodes which are not blocked, calculate the estimated cost to 
  the goal of the path that goes through that node. (This is the actual cost 
  to reach that node from the origin, plus the heuristic cost to the destination.)
- Push all those nonblocked surrounding nodes onto the Open list, and repeat loop.

In the end, the nodes along the chosen path, including the starting and ending 
position, are called the waypoints. 

Implementation Thoughts
- Use a priority queue (OOTB) for the open list.
  The heapq module provides a min heap implementation that can be used as a 
  priority queue. h[0] is always the smallest. 
- Use Manhattan distance for 4 way movement.
- For the animation, calculate the path. This builds up a list of way points.
  then step through the waypoints. The path will need to be done in a single frame
  or before the animation starts.
- https://www.gamedeveloper.com/programming/toward-more-realistic-pathfinding
  Recommends not using A* for distances greater than 40 grid times.
  the distance between origin and destination be constrained to 40 tiles, and 
  that the total search space be no more than 60x60 tiles

Per Wikipedia: https://en.wikipedia.org/wiki/A*_search_algorithm#Implementation_details
When a path is required at the end of the search, it is common to keep with each 
node a reference to that node's parent. At the end of the search these references 
can be used to recover the optimal path. If these references are being kept then
it can be important that the same node doesn't appear in the priority queue more
than once (each entry corresponding to a different path to the node, and each
with a different cost). 

A standard approach here is to check if a node about to
be added already appears in the priority queue. If it does, then the priority and
parent pointers are changed to correspond to the lower cost path. A standard 
binary heap based priority queue does not directly support the operation of 
searching for one of its elements, but it can be augmented with a hash table
that maps elements to their position in the heap, allowing this decrease-priority
operation to be performed in logarithmic time. Alternatively, a Fibonacci heap 
can perform the same decrease-priority operations in constant amortized time.

Bugs
Currently the path is just the last node.

Key things missing from my implementation.
- I need to handle the scenario that no path is found. Return a tuple? 
  Null object pattern with the Path object (e.g. NullPath, NoPath)?
- I'm not tracking the lineage of the rooms. (e.g. NewCell.parent = lastCell)
  How should I do this? I don't want to dirty up the defintion of a MazeCell
  with path traversal information. Especially since I'm using the same MazeCells
  for multiple traversal strategies. 
  - Decorator pattern?
  - Have a separete abstract data type to represent a path node (e.g. PathNode, Waypoint)?
  - A separete node object could point back to the MazeCell or store a location.
  - I should draw this abstraction out using LucidCharts or whiteboard.

- [X] I should really encapsulate the priority queue.
  - [ ] I'm not checking if a cell is already in the open list before adding it.
    I need to do this because there may be multiple ways to get to a node.
  - [ ] I don't update the total cost of cells in the open list. I need to do this 
    because there may be multiple ways to get to a node.
- [ ] I don't attempt to build the final path based on the lineage (Node.parent).
- [ ] I don't handle revisiting nodes. 
- [ ] I'm using visited_locations:Set and possible_steps: list[Point] (Min Heap, priority queue.
  The literature uses open vs closed. That seems to be a better abstraction since
  rooms can be revisited in the maze.

Game Programming Gems 1 (PDF)
- Simple Implementation: Page 248
- Optimized Implementation: Page 279 
- Fuzzy Logic for Video Games: Page 313
- A Neural Net Primer: Page 324
"""
pass

In [91]:
class Waypoint:
  """A decorator class that wraps a Point to enable chaining points."""
  def __init__(self, point: Point, predecessor: Waypoint = None):
    self._point = point
    self._predecessor = predecessor
    self._cost_from_start = 0
    self._cost_to_target = 0

  @property
  def point(self) -> Point:
    return self._point

  @property
  def predecessor(self) -> Point:
    return self._predecessor
  
  @predecessor.setter
  def predecessor(self, predecessor: Point) -> None:
    self._predecessor = predecessor

  @property 
  def cost_from_start(self) -> None:
    return self._cost_from_start
  
  @cost_from_start.setter 
  def cost_from_start(self, cost: float) -> None:
    self._cost_from_start = cost
  
  @property 
  def cost_to_target(self) -> None:
    return self._cost_to_target
  
  @cost_to_target.setter 
  def cost_to_target(self, cost: float) -> None:
    self._cost_to_target = cost
    
  def total_cost(self) -> float:
    return self.cost_from_start + self.cost_to_target

  def __str__(self):
    pred_str = f'({self._predecessor.x}, {self._predecessor.y})' if self._predecessor else 'None'
    return f'Waypoint (x = {self.point.x}, y = {self.point.y}) with predecessor {pred_str}'

  def __repr__(self) -> str:
    return self.__str__()

  def __eq__(self, other: object) -> bool:
    """For equality checks, only consider the decorated point, not the predecessor."""
    if (isinstance(other, Waypoint)):
      return self.point.x == other.point.x and self.point.y == other.point.y
    return False

  def __hash__(self) -> int:
    return self.point.__hash__()

class Path:
  def __init__(self):
    self._waypoints: list[Point] = []

  def add_waypoint(self, step: Point) -> None:
    self._waypoints.append(step)

  @property
  def waypoints(self) -> List[Point]:
    return self._waypoints

# A point stored in a heap.
# An entry is of the form (cost, count, Waypoint). 
# Not using tuples because the point can be replaced with REMOVED_POINT
PriorityPoint = List[Union[float, int, Waypoint, str]]

# A constant that represents a point that has been removed from the queue.
# I think this could lead to subtle bugs.
REMOVED_POINT = 'REMOVED'

class PriorityQueue:
  """A priority queue implemented with a min heap."""
  def __init__(self):
    self._items: List[PriorityPoint] = [] # A min heap.
    self._index: Dict[Point, PriorityPoint] = {} # An index of the points in the heap.
    self._counter = itertools.count() # A counter for tracking the sequence of points.

  def __str__(self) -> str:
    return self._items.__str__()

  def push(self, point: Waypoint, cost: float) -> PriorityQueue:
    """
    Add a point to the priority queue. Points are arranged in the queue 
    by their associated cost. The item with the smallest cost is listed first.
    If a point is already in the queue, it is removed first before adding it.

    Returns
    The instance of the priority queue.
    """

    if point in self._items:
      self.remove(point)

    count = next(self._counter)
    entry = [cost, count, point]
    self._index[point] = entry
    heappush(self._items, entry) 
    return self

  def pop(self) -> Tuple[float, Waypoint]:
    """
    Removes the point in the queue with the smallest cost.

    Returns
    A tuple of the cost and point.

    Throws
    Raises a KeyError if called on an empty queue.
    """

    # There could be removed points, so keep popping until a point is found.
    while len(self._items) > 0:
      cost,_ignore,point = heappop(self._items)
      if point is not REMOVED_POINT:
        return (cost, point)
    # If the queue is exhausted, then throw an exception.
    raise KeyError('Cannot pop from an empty priority queue.')

  def __contains__(self, point: Waypoint) -> bool:
    """
    Determines if a point is already in the queue.

    Example
    p in queue
    """
    return point in self._index

  def __len__(self) -> int:
    """
    Supports using the len() with the priority queue.
    
    Returns
    The length of the queue.
    """
    return len(self._items)

  def remove(self, point: Waypoint) -> PriorityQueue:
    """
    Removes a point from the queue if it exists. Does nothing if the point 
    doesn't exist in the queue.

    Returns
    The instance of the priority queue.
    """
    if point in self._index:
      entry = self._index.pop(point)
      entry[2] = REMOVED_POINT

In [100]:
# Testing the Queue out. REMOVE
q = PriorityQueue()
q.push(Waypoint(Point(14,22)), 13.7)
q.push(Waypoint(Point(9,2)), 3.2)
q.push(Waypoint(Point(102,14)), 32)
print(q)

assert Waypoint(Point(9,2) in q), "The point should have been found in the queue."

q.remove(Waypoint(Point(9,2)))
print(q)
item = q.pop()
print(item)

[[3.2, 1, Waypoint (x = 9, y = 2) with predecessor None], [13.7, 0, Waypoint (x = 14, y = 22) with predecessor None], [32, 2, Waypoint (x = 102, y = 14) with predecessor None]]
[[3.2, 1, 'REMOVED'], [13.7, 0, Waypoint (x = 14, y = 22) with predecessor None], [32, 2, Waypoint (x = 102, y = 14) with predecessor None]]
(13.7, Waypoint (x = 14, y = 22) with predecessor None)


In [93]:

def find_distance(a: Point, b: Point) -> float:
  """Finds the Manhattan distance between two locations."""
  return abs(a.x - b.x) + abs(a.y - b.y)

def build_path(endpoint: Waypoint) -> List[Point]:
  """
  Given a Waypoint, builds a path following the waypoint's parent pointers.
  Note: Will fail if there is a loop.

  Returns
  A path instance.
  """
  points : List[Point] = []

  current = endpoint
  while current != None:
    points.append(current.point)
    current = current.predecessor

  points.reverse()
  return points

def find_path(agent: Agent, maze: Maze, target: Point) -> Tuple[bool,Union[None,Path]]:
  """
  Finds a path from the agents current location to the target cell.

  Returns
  A tuple of the form (success:bool, Path)
  """
  visited_locations: Set[Point] = set()
  possible_steps: PriorityQueue = PriorityQueue()

  starting_point = Waypoint(agent.location, None)
  starting_point.cost_from_start = 0
  starting_point.cost_to_target = find_distance(starting_point.point, target)
  possible_steps.push(starting_point, starting_point.total_cost())

  while len(possible_steps) > 0:
    _ignore_cost, current_location = possible_steps.pop() # pop returns Tuple[float, Waypoint]
    if current_location.point == target:
      return (True, build_path(current_location))
    else:
      visited_locations.add(current_location.point)
      current_cell: MazeCell = maze.cell(current_location.point)
      
      if(current_cell is None):
        print(f'The target is ({target.x},{target.y}). Current location is ({current_location.x},{current_location.y})')
        raise Exception(f'{current_location.x},{current_location.y} has no location.')

      open_directions: list[Direction] = current_cell.open_sides()
      for direction in open_directions:
        # Find the "room" in the open direction
        neighbor_point: Point = maze.find_adjacent_neighbor(direction, current_location.point)
        neighbor = Waypoint(neighbor_point, current_location)

        # Ignore the connected room if that's where we just came from
        if (current_location.predecessor is not None) and (neighbor.point == current_location.predecessor.point) :
          continue

        neighbor.cost_from_start = find_distance(neighbor.point, starting_point.point)
        neighbor.cost_to_target = find_distance(neighbor.point, target)

        # We could have visited this room before from a different path. 
        # If that's the case, then remove it from the visited set or possible queue.
        if neighbor.point in visited_locations:
          visited_locations.remove(neighbor.point)

        if neighbor in possible_steps:
          possible_steps.remove(neighbor) 
        
        possible_steps.push(neighbor, neighbor.total_cost())
  
  return (False, None)

def draw_path(path: List[Point], canvas: Canvas) -> None:
  canvas_points = []

  # Create an array of tuples for canvas to render in a single draw call.
  print(f'Path has {len(path)} steps')
  for location in path:
    # Find the upper left corner for the room.
    upper_left_corner = Corner(location.x * ROOM_SIZE_WIDTH, location.y * ROOM_SIZE_HEIGHT)

    # Find the midpoint of the room.
    horizontal_offset = ROOM_SIZE_WIDTH/2.0
    vertical_offset = ROOM_SIZE_HEIGHT/2.0
    midpoint = Point(upper_left_corner.x + horizontal_offset, upper_left_corner.y + vertical_offset)

    canvas_points.append((midpoint.x, midpoint.y))
  
  canvas.stroke_style = 'red'
  canvas.stroke_lines(canvas_points)

In [94]:
# Test Build Path
a = Waypoint(Point(1,1))
b = Waypoint(Point(2,2), a)
c = Waypoint(Point(3,3), b)
d = Waypoint(Point(4,4), c)
e = Waypoint(Point(5,5), d)
f = Waypoint(Point(6,6), e)
g = Waypoint(Point(7,7), f)
end = Waypoint(Point(8,8), g)

path = build_path(end)
print(path)

[Point(x=1, y=1), Point(x=2, y=2), Point(x=3, y=3), Point(x=4, y=4), Point(x=5, y=5), Point(x=6, y=6), Point(x=7, y=7), Point(x=8, y=8)]


In [96]:
# 1000/125 = 8 FPS
SLEEP_TIME_SEC:float = 0.125

# Generate a maze.
maze: Maze = Maze(20, 20)
generate_maze_walls(maze)
mc = MultiCanvas(n_canvases=3, width=800, height=400)
display(mc)

# Create NPCs with different strategies
common_starting_point = Point(int(maze.width/2), int(maze.height/2))
wall_follower = Agent('blue')
wall_follower.maze_strategy(wall_follower_walk)
wall_follower.move_to(common_starting_point)
wall_follower.face(Direction.SOUTH)

random_walker = Agent('green')
random_walker.maze_strategy(random_walk)
random_walker.move_to(common_starting_point)
random_walker.face(Direction.SOUTH)

a_star_walker = Agent('yellow')
# a_star_walker.maze_strategy(a_star)
a_star_walker.move_to(common_starting_point)
a_star_walker.face(Direction.SOUTH)

# Calculate a path using A*
found_path, escape_path = find_path(a_star_walker, maze, maze.exit_cell.location)
if not found_path:
  raise Exception('Failed to find a path.') 

agents = [wall_follower, random_walker]

# Initial Render
draw_maze(maze, mc[0])
draw_path(escape_path, mc[1])
draw_agents(agents, mc[2])

# TODO: Draw the A* path as a static line.
# Does the canvas support multiple layers? Perhaps I can just draw the maze once
# on one layer, the A* path on another, and the agents on a third.
# https://github.com/martinRenou/ipycanvas/blob/master/examples/MultiCanvas.ipynb

for i in range(200):
  for agent in agents:
    agent.explore(maze)
  draw_agents(agents, mc[2])
  time.sleep(SLEEP_TIME_SEC)

Exit cell: Point(x=15, y=19)


MultiCanvas(height=400, width=800)

Path has 49 steps


In [ ]:
# The Main Entry Point
"""
SAVE_ANIMATED_GIF = False

display(out)
canvas = Canvas(width=200, height=200, sync_image_data=True)
renderer = Renderer(canvas, SAVE_ANIMATED_GIF)
thread = RenderingThread(renderer, SAVE_ANIMATED_GIF)

display(canvas)
thread.start()
"""